# Fine-tuning SWIN Transformer for Multi-label Image Classification

In [1]:
import torch.nn as nn
import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
# from transformers import DeiTForImageClassification, DeiTFeatureExtractor
import wandb
from PIL import Image
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as T
from pathlib import Path
from PIL import Image
import datasets
import os
import json
import jsonlines
from timm import list_models, create_model
from transformers.optimization import get_cosine_schedule_with_warmup
import evaluate
from torch.nn.functional import one_hot
from torch.nn.utils.rnn import pad_sequence


d:\MA\food-datasets\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

In [2]:
annotations_path = Path("../data/ingredients-101/Annotations/")
images_path = Path("../data/food-101/images/")
ingredients_path = 'ingredients_simplified.txt'
simplifications_path = Path("../data/ingredients-101/ingredients_simplification/")

In [3]:
def read_data(path, file):
    with open(path / file, 'r') as f:
        data = f.read().strip().split('\n')
    return data

In [4]:
train_images = read_data(annotations_path, 'train_images.txt')
val_images = read_data(annotations_path, 'val_images.txt')
test_images = read_data(annotations_path, 'test_images.txt')

train_labels = read_data(annotations_path, 'train_labels.txt')
val_labels = read_data(annotations_path, 'val_labels.txt')
test_labels = read_data(annotations_path, 'test_labels.txt')

base_ingredients = read_data(simplifications_path, 'baseIngredients.txt')
ingredients_simplified = read_data(annotations_path, ingredients_path)

In [5]:
train_images[:3]

['apple_pie/1005649', 'apple_pie/1014775', 'apple_pie/1026328']

In [6]:
train_labels[:3]

['0', '0', '0']

### Convert Ingredients List to Multi-label Format

In [7]:
ingredients_simplified[:3]

['butter,flour,sugar,brown sugar,apple,cinnamon,nut',
 'baby back ribs,apple,salt,mustard,brown sugar,worcestershire,gin,chili',
 'nut,cinnamon,bread,butter,phyllo dough,sugar,honey,lemon,baklava']

In [8]:
corrected_ingredients_simplified = [ingredient.strip() for sublist in ingredients_simplified for ingredient in sublist.split(',')]
corrected_ingredients_simplified[:10]

['butter',
 'flour',
 'sugar',
 'brown sugar',
 'apple',
 'cinnamon',
 'nut',
 'baby back ribs',
 'apple',
 'salt']

In [9]:
# get only the unique ingredients
unique_ingredients = list(set(corrected_ingredients_simplified))
num_unique_ingredients = len(unique_ingredients)
num_unique_ingredients

227

In [10]:
# sort the ingredients alphabetically
unique_ingredients.sort()

# create a dictionary with the ingredients as keys and the index as values
ingredient_to_idx = {ingredient: idx for idx, ingredient in enumerate(unique_ingredients)}
idx_to_ingredient = {idx: ingredient for ingredient, idx in ingredient_to_idx.items()}

# create the directory if it does not exist
os.makedirs('../data/food-ingredients', exist_ok=True)

# save to a file
with open('../data/food-ingredients/ingredient_to_idx.txt', 'w') as f:
    for key, value in ingredient_to_idx.items():
        f.write(f'{key}, {value}\n')

In [11]:
numerical_ingredients_simplified = [[ingredient_to_idx[ingredient.strip()] for ingredient in sublist.split(',')] for sublist in ingredients_simplified]

numerical_ingredients_simplified[:5]

[[36, 100, 206, 34, 5, 63, 143],
 [10, 5, 183, 141, 34, 224, 105, 54],
 [143, 63, 29, 36, 160, 206, 116, 124, 15],
 [21, 124, 105, 183, 159, 9, 7],
 [94, 204, 105, 190, 156, 42, 224, 91, 27, 78]]

In [12]:
# manually check the correctness of the conversion
print(idx_to_ingredient[183])

print([idx_to_ingredient[idx] for idx in numerical_ingredients_simplified[1]]) 

salt
['baby back ribs', 'apple', 'salt', 'mustard', 'brown sugar', 'worcestershire', 'gin', 'chili']


In [13]:
# Create a dictionary to map each class to its ingredients
class_to_ingredients = {i: ingredients for i, ingredients in enumerate(ingredients_simplified)}
class_to_ingredients

# save to a file
with open('../data/food-ingredients/class_to_ingredients.jsonl', 'w') as f:
    for key, value in class_to_ingredients.items():
        f.write(json.dumps({key: value}) + '\n')

In [14]:
# get longest ingredient list
max_len = max([len(sublist) for sublist in numerical_ingredients_simplified])
max_len

25

### Create Dataset

In [15]:
# create a jsonl file with the data

# def write_combined_data(file_path, set):
#     with jsonlines.open(file_path, 'w') as writer:
#         for image, label, ingredients in zip(f"{set}_images", f"{set}_labels", numerical_ingredients_simplified):
#             # train_images, train_labels, numerical_ingredients_simplified):
#             ingredients = class_to_ingredients[int(label)]
#             ingredients_numeric = [ingredient_to_idx[ingredient] for ingredient in ingredients.split(',')]
#             writer.write({'image': str(images_path) + '/' + image + '.jpg', 'ingredients': ingredients_numeric, 'ingredients_names': ingredients.split(','), 'class': int(label), 'class_name': image.split('/')[0]})
    
# write_combined_data('../data/food-ingredients/train.jsonl', 'train')
# write_combined_data('../data/food-ingredients/val.jsonl', 'val')
# write_combined_data('../data/food-ingredients/test.jsonl')+ 'test'
with jsonlines.open('../data/food-ingredients/train.jsonl', 'w') as writer:
    for image, label in zip(train_images, train_labels):
        ingredients = class_to_ingredients[int(label)]
        ingredients_numeric = [ingredient_to_idx[ingredient] for ingredient in ingredients.split(',')]
        writer.write({'image': str(images_path) + '/' + image + '.jpg', 'ingredients': ingredients_numeric, 'ingredients_names': ingredients.split(','), 'class': int(label), 'class_name': image.split('/')[0]})

with jsonlines.open('../data/food-ingredients/val.jsonl', 'w') as writer:
    for image, label in zip(val_images, val_labels):
        ingredients = class_to_ingredients[int(label)]
        ingredients_numeric = [ingredient_to_idx[ingredient] for ingredient in ingredients.split(',')]
        writer.write({'image': str(images_path) + '/' + image + '.jpg', 'ingredients': ingredients_numeric, 'ingredients_names': ingredients.split(','), 'class': int(label), 'class_name': image.split('/')[0]})

with jsonlines.open('../data/food-ingredients/test.jsonl', 'w') as writer:
    for image, label in zip(test_images, test_labels):
        ingredients = class_to_ingredients[int(label)]
        ingredients_numeric = [ingredient_to_idx[ingredient] for ingredient in ingredients.split(',')]
        writer.write({'image': str(images_path) + '/' + image + '.jpg', 'ingredients': ingredients_numeric, 'ingredients_names': ingredients.split(','), 'class': int(label), 'class_name': image.split('/')[0]})

In [16]:
# load dataset
data_files = {"train": "../data/food-ingredients/train.jsonl", "val": "../data/food-ingredients/val.jsonl", "test": "../data/food-ingredients/test.jsonl"}

dataset = datasets.load_dataset('json', data_files=data_files)

Generating train split: 68175 examples [00:00, 920932.17 examples/s]
Generating val split: 7575 examples [00:00, 585127.77 examples/s]
Generating test split: 25250 examples [00:00, 929107.49 examples/s]


In [17]:
# def pad_labels(batch):
#     # Convert lists to tensors and pad shorter lists
#     if len(batch['ingredients']) > 0:
#         # Convert each scalar to a 1D tensor
#         batch['ingredients'] = pad_sequence([torch.tensor([x]) for x in batch['ingredients']], batch_first=True)
#     else:
#         # Replace empty list with tensor of zeros
#         batch['ingredients'] = torch.zeros(1, max_len)  # Replace 'desired_length' with the length you want
#     return batch

# dataset = dataset.map(pad_labels)

### Visual Inspection

In [18]:
# def show_samples(ds,rows,cols):
#     # selecting random images with randomstate
#     samples = ds.shuffle(seed=20).select(range(rows*cols))
#     fig = plt.figure(figsize=(cols*4,rows*4))
#     # plotting
#     for i in range(rows*cols):
#         img = samples[i]['image']
#         # add full path and open image
#         img = Image.open(img)

#         labels = samples[i]['ingredients']
#         # getting string labels and combining them with a comma
#         labels = ','.join([idx_to_ingredient[lb] for lb in labels])
#         fig.add_subplot(rows,cols,i+1)
#         plt.imshow(img)
#         plt.title(labels)
#         plt.axis('off')
            
# show_samples(dataset['train'],rows=2,cols=2)

### Perform Transformations

In [19]:
img_size = (224,224)

tfms = T.Compose([
    T.Resize(img_size),
    T.RandomHorizontalFlip(),
    T.RandomRotation(30),
    T.CenterCrop(img_size),
    T.ToTensor(),
    T.Normalize(
        mean = (0.5,0.5,0.5),
         std = (0.5,0.5,0.5)
    )
])

valid_tfms = T.Compose([
    T.Resize(img_size),
    T.ToTensor(),
    T.Normalize(
        mean = (0.5,0.5,0.5),
         std = (0.5,0.5,0.5)
    )
])


In [20]:
def transforms(batch):
    for i in range(len(batch['image'])):
        img = Image.open(batch['image'][i]).convert('RGB')
        img = tfms(img)
        batch['image'][i] = img
    return batch

def transforms_valid(batch):
    for i in range(len(batch['image'])):
        img = Image.open(batch['image'][i]).convert('RGB')
        img = valid_tfms(img)
        batch['image'][i] = img
    return batch

In [21]:
# Apply the transformations to the dataset
train_dataset = dataset['train'].with_transform(transforms)
valid_dataset = dataset['val'].with_transform(transforms_valid)
test_dataset = dataset['test'].with_transform(transforms)

len(train_dataset), len(valid_dataset), len(test_dataset)

(68175, 7575, 25250)

In [22]:
train_dataset

Dataset({
    features: ['image', 'ingredients', 'ingredients_names', 'class', 'class_name'],
    num_rows: 68175
})

In [23]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Separate the inputs and labels
    inputs = [item['image'] for item in batch]
    labels = [item['ingredients'] for item in batch]

    # Pad the labels to the same length
    labels = pad_sequence([torch.tensor(label) for label in labels], batch_first=True)

    # Convert to one-hot encoding
    one_hot_labels = torch.zeros((len(labels), num_unique_ingredients))
    for i, label in enumerate(labels):
        one_hot_labels[i, label] = 1

    # Stack the inputs into a tensor
    inputs = torch.stack(inputs)

    return {'image': inputs, 'ingredients': one_hot_labels}


# train_loader = DataLoader(dataset, batch_size=8, collate_fn=collate_fn)

In [24]:
# # uncomment for testing with less data

# # Anzahl der zu verwendenden Samples festlegen
# from sklearn.model_selection import train_test_split
# from torch.utils.data import Subset


# subset_size = 0.1  # Nutze 10% der Daten
# train_indices, _ = train_test_split(range(len(train_dataset)), train_size=subset_size, random_state=42)
# valid_indices, _ = train_test_split(range(len(valid_dataset)), train_size=subset_size, random_state=42)
# test_indices, _ = train_test_split(range(len(test_dataset)), train_size=subset_size, random_state=42)

# # Erstellen der Subsets
# train_subset = Subset(train_dataset, train_indices)
# valid_subset = Subset(valid_dataset, valid_indices)
# test_subset = Subset(test_dataset, test_indices)

In [25]:
# create DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

In [26]:
# check size of the data
len(train_loader), len(valid_loader), len(test_loader)

(8522, 947, 3157)

## Train Model

In [27]:
os.environ["WANDB_NOTEBOOK_NAME"] = 'swin-fine-tuning-multi-label.ipynb'

In [28]:
import time

from tqdm import tqdm

def train(model_name, batch_size=8, epochs=1, lr=2e-4):
    os.environ["WANDB_DISABLE_SYMLINKS"] = "1"
    wandb.init(project=model_name+"-finetuning", config={
        "model_name": model_name,
        "batch_size": batch_size,
        "epochs": epochs,
        "learning_rate": lr
    })
    config = wandb.config
    
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    print(f'Using {device}')

    # Debug: Model Load
    print("Loading the model...")
    model = create_model(model_name, num_classes=num_unique_ingredients)
    model.to(device)
    print("Model loaded.")

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.BCEWithLogitsLoss()
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * len(train_loader)),
        num_training_steps=len(train_loader) * epochs
    )

    for epoch in range(epochs):
        model.train()
        train_metric = evaluate.load('roc_auc', 'multilabel')
        running_loss = 0.0

        print(f"Starting training epoch {epoch}...")

        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
            inputs = batch['image'].to(device)
            labels = batch['ingredients'].to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            running_loss += loss.item()
            train_metric.add_batch(references=labels.cpu(), prediction_scores=outputs.cpu())
        
        
        train_loss = running_loss / len(train_loader)
        train_roc_auc = train_metric.compute(average='micro')['roc_auc']
        
        print(f"\n{epoch = }")
        print(f"{train_loss = :.3f} | {train_roc_auc = :.3f}")
        
        wandb.log({"train_loss": train_loss, "train_roc_auc": train_roc_auc, "epoch": epoch})
        
        model.eval()
        running_loss = 0.0
        valid_metric = evaluate.load('roc_auc', 'multilabel')
        
        print(f"Starting validation epoch {epoch}...")
        val_start_time = time.time()
        
        with torch.no_grad():
            for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch}"):
                inputs = batch['image'].to(device)
                labels = batch['ingredients'].to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                running_loss += loss.item()
                valid_metric.add_batch(references=labels.cpu(), prediction_scores=outputs.cpu())
        
        val_end_time = time.time()
        print(f"Completed validation epoch {epoch} in {val_end_time - val_start_time:.2f} seconds")
        
        valid_loss = running_loss / len(valid_loader)
        valid_roc_auc = valid_metric.compute(average='micro')['roc_auc']
        
        print(f"{valid_loss = :.3f} | {valid_roc_auc = :.3f}")
        
        wandb.log({"valid_loss": valid_loss, "valid_roc_auc": valid_roc_auc, "epoch": epoch})
        
        model_path = f'./models/{model_name}-pascal-epoch-{epoch}.pth'
        torch.save(model.state_dict(), model_path)
    
    test_metric = evaluate.load('roc_auc', 'multilabel')
    
    print(f"Starting test evaluation...")
    test_start_time = time.time()
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            inputs = batch['image'].to(device)
            labels = batch['ingredients'].to(device)
            outputs = model(inputs)
            test_metric.add_batch(references=labels.cpu(), prediction_scores=outputs.cpu())
    
    test_end_time = time.time()
    print(f"Completed test evaluation in {test_end_time - test_start_time:.2f} seconds")
    
    test_roc_auc = test_metric.compute(average='micro')['roc_auc']
    print(f"\n\nTEST AUROC: {test_roc_auc:.3f}")
    
    wandb.log({"test_roc_auc": test_roc_auc})

    return model

In [29]:
model_name = 'swin_s3_base_224'
# model_name = 'resnet18'
# train the model
model = train(model_name, batch_size=8, epochs=5, lr=2e-4)

wandb: Currently logged in as: ahirsch (htwbe). Use `wandb login --relogin` to force relogin


Using cuda:1
Loading the model...
Model loaded.
Starting training epoch 0...


Training Epoch 0: 100%|██████████| 8522/8522 [42:42<00:00,  3.33it/s]  



epoch = 0
train_loss = 0.139 | train_roc_auc = 0.815
Starting validation epoch 0...


Validation Epoch 0: 100%|██████████| 947/947 [01:37<00:00,  9.76it/s]


...

Processed test batch in 0.06 seconds
Processed test batch in 0.06 seconds


Testing: 100%|█████████▉| 3155/3157 [07:32<00:00,  7.12it/s]

Processed test batch in 0.06 seconds
Processed test batch in 0.06 seconds


Testing: 100%|██████████| 3157/3157 [07:32<00:00,  6.97it/s]


Processed test batch in 0.06 seconds
Completed test evaluation in 452.63 seconds


TEST AUROC: 0.814
